In [174]:
from torch import nn
import torch
import pandas as pd

from torch.optim import Adam

from sklearn.preprocessing import (
    StandardScaler,
    RobustScaler,
    PolynomialFeatures,
    SplineTransformer,
    OneHotEncoder,
)
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.metrics import make_scorer, r2_score
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor
from sklearn.base import BaseEstimator
from sklearn.model_selection import KFold, cross_validate
from sklearn.cluster import KMeans

from plotnine import (
    ggplot,
    aes,
    geom_histogram,
    facet_wrap,
    geom_point,
    geom_smooth,
    geom_bar,
)

torch.set_default_device("mps")

In [175]:
train = pd.read_csv("../data/train.csv", index_col="SEQN")
test = pd.read_csv("../data/test.csv", index_col="SEQN")
y = train.pop("y")
train.head()

,self_eval,teacher_eval,extracurricular,district,SRP_1,SRP_2,SRP_3,SRP_4,SRP_5,SRP_6,...,SRP_41,SRP_42,SRP_43,SRP_44,SRP_45,SRP_46,SRP_47,SRP_48,SRP_49,SRP_50
SEQN,,,,,,,,,,,,,,,,,,,,,
969167,4,5,9,3,-0.181,-0.379,-0.164,0.080,0.378,1.581,...,2.070,-1.156,-0.730,-0.508,-0.497,0.224,0.412,-0.517,0.099,0.114
188942,4,3,5,4,-0.126,1.603,1.021,0.489,-1.404,-0.955,...,1.478,-0.318,1.240,-1.993,2.021,-1.078,-0.277,0.802,0.253,-0.720
134058,1,2,8,5,0.724,-0.702,2.249,0.910,0.330,0.411,...,0.119,0.449,1.980,-0.401,-0.544,-0.944,1.592,0.875,-0.734,-2.336
124022,3,3,10,6,0.706,-0.302,1.023,-0.895,0.625,1.283,...,1.249,2.025,-2.289,-0.407,0.025,-0.515,0.408,1.380,-1.075,-2.451
685285,5,5,1,5,-0.350,-1.001,0.931,0.192,0.491,0.292,...,0.341,-0.118,-0.288,0.457,-0.566,0.822,-0.317,0.661,2.096,0.004


In [177]:
class RowSum(BaseEstimator):

    def __init__(self) -> None:
        super().__init__()

    def fit(self, *args, **kwargs):
        pass

    def transform(self, X, y=None, **kwargs):
        if isinstance(X, pd.DataFrame):
            return X.values.sum(axis=1).reshape(-1, 1)
        else:
            return X.sum(axis=1).reshape(-1, 1)

    def fit_transform(self, X, y=None, **kwargs):
        return self.transform(X, y, **kwargs)

    def get_feature_names_out(self, X, y=None, **kwargs):
        return ["rowSums"]


sumRows = Pipeline(
    [
        ("sums", RowSum()),
        ("scale", StandardScaler()),
    ]
)

colTrans = ColumnTransformer(
    [
        ("one_hot", OneHotEncoder(sparse_output=False, drop="first"), ["district"]),
        ("collapse", sumRows, [f"SRP_{i}" for i in range(1, 51)]),
    ],
    remainder="passthrough",
)

preprocessor = Pipeline(
    [
        ("setup", colTrans),
        ("polys", PolynomialFeatures(interaction_only=True)),
        ("splines", SplineTransformer(knots="quantile")),
    ]
)

# pd.DataFrame(out, columns=preprocessor.get_feature_names_out())

In [179]:
model_results = []
for model in [
    LinearRegression(),
    # RandomForestRegressor(n_estimators=256, n_jobs=8)
    # HistGradientBoostingRegressor(max_iter=500),
]:
    model_pipeline = Pipeline([("data", preprocessor), ("model", model)])
    res = cross_validate(model_pipeline, X=train, y=y, scoring=make_scorer(r2_score), return_estimator=True)
    model_results.append(res)

In [ ]:
final_model = Pipeline([("preprocess", preprocessor), ("model", model)]).fit(train, y=y)
test["y"] = final_model.predict(test)



In [180]:
def optim(
    learning_rate=0.1,
    max_iter=10,
    l2_regularization=1,
    max_bins=50,
    max_depth=4,
    max_leaf_nodes=5,
    return_model=False,
):

    scorer = make_scorer(r2_score)
    max_depth = int(max_depth)
    max_bins = int(max_bins)
    max_leaf_nodes = int(max_leaf_nodes)
    max_iter = int(max_iter)

    model = HistGradientBoostingRegressor(
        learning_rate=learning_rate,
        max_iter=max_iter,
        l2_regularization=l2_regularization,
        max_bins=int(max_bins),
        categorical_features=[len(train.columns) - 1],
        max_leaf_nodes=max_leaf_nodes,
        max_depth=max_depth,
    )

    model_pipeline = Pipeline([("preprocess", preprocessor), ("model", model)])

    # TODO: add cross manual cross validation here within district
    if return_model:
        return (
            cross_validate(
                model_pipeline,
                X=train,
                y=train,
                scoring=scorer,
                cv=KFold(shuffle=True),
            ),
            model_pipeline,
        )

    else:
        for _ in range(3):
            try:
                return cross_validate(
                    model_pipeline,
                    X=train_x,
                    y=train_y,
                    scoring=scorer,
                    cv=KFold(shuffle=True, n_splits=10),
                )["test_score"].mean()

            except (ValueError, TypeError):
                continue


# hist_cv
colTrans = ColumnTransformer(
    [
        # ("one_hot", OneHotEncoder(sparse_output=False, drop="first"), ["district"]),
        ("collapse", sumRows, [f"SRP_{i}" for i in range(1, 51)]),
    ],
    remainder="passthrough",
)

preprocessor = Pipeline(
    [
        ("setup", colTrans),
        # ("polys", PolynomialFeatures(interaction_only=True)),
        # ("splines", SplineTransformer(knots="quantile")),
    ]
)

preprocessor.fit_transform()